In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
!pip install pandas
!pip install pycocotools
!pip install opencv-python
!pip install albumentations

In [2]:
import zipfile

zip_path = "/content/drive/MyDrive/P2-Dhaka_Dataset_COCO/P2_Dhaka_Dataset.v29i.coco.zip"
extract_path = "/content/drive/MyDrive/P2-Dhaka_Dataset_COCO/"

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction successful!")
except zipfile.BadZipFile:
    print("Bad ZIP file: corrupted or incomplete.")

Extraction successful!


In [1]:
import torchvision
from torchvision.models.detection import retinanet_resnet50_fpn_v2

model = retinanet_resnet50_fpn_v2(weights=None,weights_backbone="DEFAULT", num_classes=9)
model.train()
model.to('cuda')

RetinaNet(
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      

In [2]:
import torch
from torchvision.datasets import CocoDetection
from torchvision import transforms as T
import os

class CocoDetectionRetinaNet(CocoDetection):
    def __init__(self, root, annFile, transform=None):
        super(CocoDetectionRetinaNet, self).__init__(root, annFile)
        self.transform = transform

    def __getitem__(self, idx):
        img, ann = super().__getitem__(idx)

        boxes = []
        labels = []

        for obj in ann:
            if 'iscrowd' in obj and obj['iscrowd']:
                continue

            bbox = obj['bbox']
            x1 = bbox[0]
            y1 = bbox[1]
            x2 = bbox[0] + bbox[2]
            y2 = bbox[1] + bbox[3]

            boxes.append([x1, y1, x2, y2])
            labels.append(obj['category_id'])

        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels
        }

        if self.transform is not None:
            img = self.transform(img)

        return img, target


In [4]:
!pip install pycocotools

In [3]:
from torchvision.datasets import CocoDetection
from torchvision.transforms import ToTensor

train_ds = CocoDetectionRetinaNet(root='P2_Dhaka_COCO/train',
                         annFile='P2_Dhaka_COCO/annotations/train_annotations.coco.json',
                         transform=ToTensor())

test_ds = CocoDetectionRetinaNet(root='P2_Dhaka_COCO/test',
                        annFile='P2_Dhaka_COCO/annotations/test_annotations.coco.json',
                        transform=ToTensor())

val_ds = CocoDetectionRetinaNet(root='P2_Dhaka_COCO/valid',
                       annFile='P2_Dhaka_COCO/annotations/val_annotations.coco.json',
                       transform=ToTensor())

loading annotations into memory...
Done (t=0.42s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [4]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_ds,
                      batch_size=5,
                      shuffle=True,
                      collate_fn=lambda batch: tuple(zip(*batch)))

test_dl = DataLoader(test_ds,
                     batch_size=5,
                     shuffle=False,
                     collate_fn=lambda batch: tuple(zip(*batch)))

val_dl = DataLoader(val_ds,
                    batch_size=8,
                    shuffle=False,
                    collate_fn=lambda batch: tuple(zip(*batch)))

In [5]:
import torch
from torch import optim as optim

In [6]:
from tqdm import tqdm

In [7]:
from torchvision.models.detection import retinanet_resnet50_fpn
import torch

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 50
curr_loss = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in tqdm(train_dl, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True):
        images = [img.to('cuda') for img in images]
        targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
        
        if any(t['boxes'].numel() == 0 for t in targets):
            continue

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
    for images, targets in tqdm(test_dl, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True):
        images = [img.to('cuda') for img in images]
        targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]
        
        if any(t['boxes'].numel() == 0 for t in targets):
            continue

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")
    if running_loss < curr_loss:
        print(f"Loss decreased from {curr_loss} to {running_loss}")
        curr_loss = running_loss
        torch.save(model.state_dict(), "retinanet_weights.pth")

Epoch 1/50: 100%|██████████| 61/61 [00:48<00:00,  1.25it/s]


Epoch 1, Loss: 530.9777
Loss decreased from 1000 to 530.9776656031609


Epoch 2/50: 100%|██████████| 61/61 [00:47<00:00,  1.27it/s]


Epoch 2, Loss: 283.7620
Loss decreased from 530.9776656031609 to 283.7620059847832


Epoch 3/50: 100%|██████████| 61/61 [00:46<00:00,  1.32it/s]


...

Epoch 48, Loss: 36.6998
Loss decreased from 36.893745025619864 to 36.69978182762861


Epoch 49/50: 100%|██████████| 61/61 [00:45<00:00,  1.34it/s]


Epoch 49, Loss: 41.0421


Epoch 50/50: 100%|██████████| 61/61 [00:45<00:00,  1.34it/s]


Epoch 50, Loss: 37.5948


In [8]:
model.eval()
with torch.no_grad():
    for images, targets in val_dl:
        images = list(img.to('cuda') for img in images)
        outputs = model(images)

In [12]:
import torch
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# Init metric
metric = MeanAveragePrecision(iou_type="bbox", iou_thresholds=[0.5])  # mAP@0.5

model.eval()
metric.reset()

with torch.no_grad():
    for images, targets in val_dl:
        images = [img.to('cuda') for img in images]
        targets = [{k: v.to('cuda') for k, v in t.items()} for t in targets]

        outputs = model(images)

        # Format prediction and targets for torchmetrics
        preds = []
        for out in outputs:
            preds.append({
                'boxes': out['boxes'].cpu(),
                'scores': out['scores'].cpu(),
                'labels': out['labels'].cpu(),
            })

        gts = []
        for t in targets:
            gts.append({
                'boxes': t['boxes'].cpu(),
                'labels': t['labels'].cpu(),
            })

        metric.update(preds, gts)

# Compute results
results = metric.compute()
print("mAP@0.5:", results["map_50"].item())


mAP@0.5: 0.7855510711669922
